# Motion correction online

This notebook uses the code for motion correction online based on the motion corretion code from Online CNMF (OnAcid) from Caiman and from the original motion correction code from NoRMCorr.

If first opens the required packages for running the code, then sets the files to perform motion correction on and sets the required parameters for performing motion correction.

In [22]:
import logging
try:
    if __IPYTHON__:
        # this is used for debugging purposes only. allows to reload classes when changed
        get_ipython().magic('load_ext autoreload')
        get_ipython().magic('autoreload 2')
except NameError:
    pass

logging.basicConfig(format=
                          "%(relativeCreated)12d [%(filename)s:%(funcName)20s():%(lineno)s] [%(process)d] %(message)s",
                    # filename="/tmp/caiman.log",
                    level=logging.INFO)

import numpy as np
import os
import caiman as cm
import mesmerize_core as mescore
from caiman.source_extraction import cnmf
import matplotlib.pyplot as plt

from fastplotlib import ImageWidget, Plot, GridPlot
from fastplotlib.graphics.line_slider import LineSlider
from ipywidgets import VBox, IntSlider, Layout

C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_24984\2272691973.py:5: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('load_ext autoreload')
C:\Users\Gebruiker\AppData\Local\Temp\ipykernel_24984\2272691973.py:6: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('autoreload 2')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Set data

Data can be set in a loop but should be set as a list and not as a string. 
Otherwise the code will turn them into a list.

Using '/' instead of '\\' allows to pass this code among operative systems

In [5]:
fnames = ["Z:/Calcium_Analysis/12937-06_S/12937-06AI_2DPI_S.tif",
         "Z:/Calcium_Analysis/12937-06_S/12937-06AI_3DPI_S.tif",
         "Z:/Calcium_Analysis/12937-06_S/12937-06AI_4DPI_S.tif",
         "Z:/Calcium_Analysis/12937-06_S/12937-06AI_6DPI_S.tif",
         "Z:/Calcium_Analysis/12937-06_S/12937-06AI_7DPI_S.tif",
         "Z:/Calcium_Analysis/12937-06_S/12937-06AI_8DPI_S.tif"]

if isinstance(fnames,str):
    fnames = [fnames]

for i in range(0,len(fnames)):
    if os.path.exists(fnames[i]) == False:
        logging.error('File '+ str(i+1) + ': '+fnames[i] + ' does not exits! Please correct the filename.')

## Set parameters

This is where the parameters are set for motion correction. These are set as CNMF parameters because the online_motion_correction code is inside the online CNMF pipeline code. This may change in future updates.

All the parameters are explained below. However, the most important parameters in this code are:

<ul>
    <li><b> gSig_filt: </b> Gaussian filter. Should normally be half the neuron size.</li>
    <li><b> pw_rigid: </b> Set True for piece-wise motion correction.</li>
    <li><b> max_shifts_online: </b> Maximum rigid shifts. It should not be huge to not have major problems in the updated templates.</li>
</ul>

In [28]:
gSig_filt = (6,6)                    #Gaussian filter size. It should normally be half the neuron size. Set it to none for 2-photon
pw_rigid = False                     #True if piece-wise motion correction should be used
strides = (300, 300)                  #Patch sizes in pixels.
overlaps = (24, 24)                 #Overlap between patches in pixels. This value normally does not have to be very high.
max_shifts_online = 40              #Maximum rigid shifts
max_deviation_rigid = 6       #Maximum deviation in pixels between rigid shifts and shifts of individual patches
border_nan = 'copy'                 #flag for allowing NaN in the boundaries. True allows NaN, whereas 'copy' copies the value of the nearest data point.
shifts_opencv = True                #flag for applying shifts using cubic interpolation (otherwise FFT)
init_batch = 500                    #initial batch 
ds_factor = 1                       #Spatial binning factor. Have in account that if spatial binning is used, all values above are unaltered.
normalize = False                    #Whether to normalize each frame prior to online processing. Set it to False for 2 photon

params_dict = {'fnames': fnames,
               'gSig_filt': gSig_filt,
               'pw_rigid': pw_rigid,
               'strides': strides,
               'overlaps': overlaps,
               'max_shifts_online': max_shifts_online,
               'max_deviation_rigid': max_deviation_rigid,
               'border_nan': border_nan,
               'shifts_opencv': shifts_opencv,
               'init_batch': init_batch,
               'ds_factor': ds_factor,
               'normalize': normalize}
opts = cnmf.params.CNMFParams(params_dict=params_dict)
cnm = cnmf.online_cnmf.OnACID(params=opts)  

    63220195 [params.py:                 set():976] [24984] Changing key fnames in group data from None to ['Z:/Calcium_Analysis/12937-06_S/12937-06AI_1DPI_S.tif']
    63220198 [params.py:                 set():976] [24984] Changing key max_shifts_online in group online from 10 to 40
    63220200 [params.py:                 set():976] [24984] Changing key init_batch in group online from 200 to 500
    63220200 [params.py:                 set():976] [24984] Changing key gSig_filt in group motion from None to (6, 6)
    63220201 [params.py:                 set():976] [24984] Changing key strides in group motion from (96, 96) to (300, 300)
    63220201 [params.py:                 set():976] [24984] Changing key overlaps in group motion from (32, 32) to (24, 24)
    63220202 [params.py:                 set():976] [24984] Changing key max_deviation_rigid in group motion from 3 to (10, 10)


# Extra variables

You can also set the precision with which you want the motion corrected images to be stored, which can save some storage space. We don't advise to store with less precision than the original recording.

You can also open a template as a numpy array if you want to match these images against a previous template. Useful for when you have two channels or sucessive recordings of the same FOV with similar signal and background.

In [7]:
#These are extra variables
nbits = np.float16                          #Number of bits in which to save final images as tiff files.
init_template = None                        #If an initial template can be provided, this should be where to set this variable into 
                                                   #the template 2D numpy array.

## Run online motion correction

This code uses the same template for 2nd and subsequent files if <b> reuse_template </b> is set to True

In [29]:
reuse_template = False
MCfnames = fnames.copy()

for i in range(0,len(fnames)):
    cnm.params.set('data',{'fnames':[fnames[i]]})
    cnm.motion_correction_online(template = init_template, save_movie=True,nbits=nbits)

    MCfnames[i] = os.path.join(os.path.dirname(fnames[i]),os.path.basename(fnames[i]).split('.')[0]+'_MC Results', os.path.splitext(os.path.basename(fnames[i]))[0]+str('_MC.tif'))
    
    if reuse_template:
        init_template = cm.load(os.path.join(os.path.dirname(fnames[i]), 'MC_results', 'MC_templates.tif'))[-1]

    63223132 [online_cnmf.py:motion_correction_online():1026] [24984] Analyzing Z:/Calcium_Analysis/12937-06_S/12937-06AI_1DPI_S.tif
    63223439 [movies.py:                load():1605] [24984] Your tif file is multiseries. Performance may be affected.
    63240933 [movies.py:      extract_shifts():344] [24984] Movie average is negative. Removing 1st percentile.
    63241345 [movies.py:      extract_shifts():362] [24984] Movie average is negative. Removing 1st percentile.
    63261943 [movies.py:      extract_shifts():344] [24984] Movie average is negative. Removing 1st percentile.
    63262238 [movies.py:      extract_shifts():362] [24984] Movie average is negative. Removing 1st percentile.
    63281202 [movies.py:      extract_shifts():344] [24984] Movie average is negative. Removing 1st percentile.
    63281409 [movies.py:      extract_shifts():362] [24984] Movie average is negative. Removing 1st percentile.
    63297498 [online_cnmf.py:motion_correction_online():1053] [24984] Initi

## Plot x and y shifts

Plotting shifts can allow to have an idea of if the motion correction went well. A typical example is when the maximum shifts allowed is lower than the actual shifts that occured. When that happens, it is common to see x or y shifts jumping from very big negative values to very positive values or vice-versa in a small number of frames. These jumps, however, are not common.

In [ ]:
T = len(cnm.estimates.shifts) #Calculated shifts
    
#Get x and y shifts
shifts = cnm.estimates.shifts[-T:]
if (pw_rigid is True):
    y_shifts = [[sx[1] for sx in sh] for sh in shifts]
    x_shifts = [[sx[0] for sx in sh] for sh in shifts]

else:
    y_shifts = [sh[1] for sh in shifts]
    x_shifts = [sh[0] for sh in shifts]
    
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.plot(x_shifts)
ax1.set(title = 'X shifts', xlabel = 'Frames', ylabel = 'Pixel shift')
ax2.plot(y_shifts)
ax2.set(title = 'Y shifts', xlabel = 'Frames', ylabel = 'Pixel shift')

## Check motion corrected file

Another way to check the motion correction file is by opening it and run through the frames side-by-side with the original file to see if the movement is gone.

<b> If you had a list of several files, select the index of files to compare (the first file has index=0). </b>

In [23]:
i = 0                         #Index of files to compare

originalMovie = mescore.movie_readers.default_reader(fnames[i])
MC_movie = mescore.movie_readers.default_reader(MCfnames[i])

mcorr_iw = ImageWidget(
    data=[originalMovie, MC_movie], 
    vmin_vmax_sliders=True, 
    cmap="gray"
)

mcorr_iw.show()

RFBOutputContext()

C:\Users\Gebruiker\mambaforge\envs\caimanRui\lib\site-packages\mesmerize_core\arrays\_tiff.py:98: UserWarning: min not implemented for LazyTiff, returning min of 0th index
  warn("min not implemented for LazyTiff, returning min of 0th index")
C:\Users\Gebruiker\mambaforge\envs\caimanRui\lib\site-packages\mesmerize_core\arrays\_tiff.py:103: UserWarning: max not implemented for LazyTiff, returning min of 0th index
  warn("max not implemented for LazyTiff, returning min of 0th index")


### Use mean window function to visualize motion correction better

By setting a rolling average of a number of frames, motion correctio artifacts are more easily visualized.

In [ ]:
# Use rolling mean over 10 frames in this case.
mcorr_iw.window_funcs = {"t": (np.mean, 10)}

## Close the canvas to free up GPU processing time, not necessary if you have a powerful GPU

Computers with slow GPU may not have the capability of running several things in parallel. Thus, it is important to close the image widget if it was opened before

In [ ]:
mcorr_iw.plot.canvas.close()